# Coarse Graining DS from an interface

## First we need to take care of imports

In [1]:
import sys, os, copy
print('python path:\n    {}'.format(sys.path))
import sim, pickleTraj
import subprocess
from datetime import datetime
#import my special md-manager utilities
from utility import *
print('... execution date: {}'.format(datetime.now()))
print(context.get_devinfo(sim))
print('... cwd:\t{}'.format(os.getcwd()))


import numpy as np
from collections import OrderedDict 
from collections import namedtuple
units = sim.units.DimensionlessUnits

python path:
    ['/home/kshen/CGmodel/MD/L30-15-15/newsim/SDS_fresh/cg_SDS/a.25C/0a.onepot__C12W.0e/1.newtraj/1.srelrun_correctFilter/2a.matchRee_matchAz_manual3', '', '/home/kshen/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles', '/home/kshen/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles/lib/python', '/home/kshen/anaconda3/envs/py27dev/lib/python27.zip', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7/plat-linux2', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7/lib-tk', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7/lib-old', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7/lib-dynload', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7/site-packages', '/home/kshen/mylib/sim.scout', '/home/kshen/mylib/packmol', '/home/kshen/mylib/pypfts', '/home/kshen/anaconda3/envs/py27dev/lib/python2.7/site-packages/IPython/extensions', '/home/kshen/.ipython']
default recursion limit

## Take care of coarse-graining parameters

In [2]:
sim.export.omm.platformName = 'OpenCL'
sim.export.omm.device = 1 #-1 is default, let openmm choose its own platform. otherwise is GPU device #
sim.export.omm.NPairPotentialKnots = 500 #number of points used to spline-interpolate the potential
sim.export.omm.InnerCutoff = 0.001 #0.001 is default. Note that a small value is not necessary, like in the lammps export, because the omm export interpolates to zero
sim.srel.optimizetrajomm.OpenMMStepsMin = 0 #number of steps to minimize structure, 0 is default
sim.srel.optimizetrajomm.OpenMMDelTempFiles = False #False is Default
sim.export.omm.UseTabulated = False
#sim.export.omm.barostatFreq = 10
sim.srel.base.n_process = 8
print('NUM PROCESSES: {}'.format(sim.srel.base.n_process))

sim.export.omm.VVerbose = False
sim.srel.base.VVerbose = False
sim.srel.penalty.VVerbose = False

UseWPenalty = False
StageCoefs = []
MaxIter=None
#MaxIter=1
SteepestIter=0

# === special_options that override inputs, i.e. neutral system, recalculation options ===
srel_on_neutral_sys = False
relax  = False
recalc = False
#struct_vars = None #expected syntax: list of tuples, (name of potential, name of variable)
#                   I set this by calling ff.get_free_parameters below...
thermo_vars = [('Uext','UConst')]


NUM PROCESSES: 8


## Specify system data sources

In [3]:
Uext = 0.0
#... other inputs
datadir='.'
traj_file = os.path.abspath('{}/SDS-C12W-2MNaCl-25C_mapped.lammpstrj.gz'.format(datadir))
forcefield_file = './setup/ff_initial.dat' #in sim format
traj = pickleTraj(traj_file)
box = traj.FrameData['BoxL']

## Create and set up the system

In [4]:
print('===== Defining system and simulation =====')
top = topologify.Topology('./setup/top.yaml')

options0 = yaml.load('./setup/settings_bondljg.yaml')
options0['box'] = box


options1 = yaml.load('./setup/settings_bondonly.yaml')
options1['box'] = box

options1proxy = yaml.load('./setup/settings_bondonly.yaml')
options1proxy['box'] = box
options1proxy['sys_name'] = 'bondonly_proxy' #this is the proxy sys for bondonly CG

ffdef0 = forcefield.ForceField('./setup/ff_bondljg.yaml')
ffdef1 = forcefield.ForceField('./setup/ff_bondonly.yaml')

#make Sys0, for doing ljg srel
sysname_prefix = options0['sys_name']
options0['neutralize'] = True
Sys0 = export_sim.create_system(top, options=options0)
export_sim.set_system_settings( Sys0, options0 )
ffs0 = export_sim.create_forcefield( Sys0,top,ffdef0, options=options0 )
Sys0.ForceField.extend(ffs0)

options0['neutralize'] = False
options0['sys_name'] = sysname_prefix + '_elec'
ElecSys = export_sim.create_system(top, options=options0)
export_sim.set_system_settings( ElecSys, options0 )
Elecffs = export_sim.create_forcefield( ElecSys,top,ffdef0, options=options0 )
ElecSys.ForceField.extend(Elecffs)


#make Sys1, for doing bond constraint 
sysname_prefix = options1['sys_name']
Sys1 = export_sim.create_system(top, options=options1)
export_sim.set_system_settings( Sys1, options1 )
ffs1 = export_sim.create_forcefield( Sys1,top,ffdef1, options=options1 )
Sys1.ForceField.extend(ffs1)

options0['sys_name'] = sysname_prefix + '_elec4bondonlyproxy'
Sys1proxy = export_sim.create_system(top, options=options0)
export_sim.set_system_settings( Sys1proxy, options0 )
ffs1proxy = export_sim.create_forcefield( Sys1proxy,top,ffdef0, options=options0 )
Sys1proxy.ForceField.extend(ffs1proxy)


print('=== Adding measures ===')
Az = sim.measure.Az(Sys0,axis=2)
Az_target = 18.8556366
Sys0.Measures.append(Az)
sim.srel.penalty.VVerbose = False

sim_mol_DS = [mol for mol in Sys1.World if mol.Name == 'DS'][0]
ST1,ST2 = Sys1.World.SiteTypes[sim_mol_DS[1].SID], Sys1.World.SiteTypes[sim_mol_DS[6].SID] 
DS_ree_filter = sim.atomselect.PolyFilter(Filters=[ST1,ST2],Intra=True)
Dist2 = sim.measure.distang.Distance2(Sys1,DS_ree_filter, Name='tail_ree2')
Sys1.Measures.append(Dist2)
Dist2_target = 1.0


print('=== Loading and locking (compiling) system ===')
# load systems
export_sim.load_system(Sys0, forcefield_file)
export_sim.load_system(Sys1proxy, forcefield_file)
export_sim.load_system(Sys1, forcefield_file)
#struct_vars = export_sim.get_free_params(Sys.ForceField)



===== Defining system and simulation =====
===== Building topology =====

=== Processing Bead Types =====
--->processing entry 0: ordereddict([('fields', ['mass', 'charge']), ('defaults', [1.0, 0.0])])
--->processing entry 1: ['W', 1.0, 0.0]
... successfully added new element W with mass 1.0
	final processed bead def: {'charge': 0.0, 'mass': 1.0, 'name': 'W'}
--->processing entry 2: ['C2', 1.0, 0.0]
... successfully added new element C2 with mass 1.0
	final processed bead def: {'charge': 0.0, 'mass': 1.0, 'name': 'C2'}
--->processing entry 3: ['SO4', 1.0, -1.0]
... successfully added new element SO4 with mass 1.0
	final processed bead def: {'charge': -1.0, 'mass': 1.0, 'name': 'SO4'}
--->processing entry 4: ['Na', 1.0, 1.0]
... successfully added new element Na with mass 1.0
	final processed bead def: {'charge': 1.0, 'mass': 1.0, 'name': 'Na'}
--->processing entry 5: ['Cl', 1.0, -1.0]
... successfully added new element Cl with mass 1.0
	final processed bead def: {'charge': -1.0, 'mass'

## Create Optimizer

In [6]:
print('===== Making optimizer =====')
Opt0 = optimizer.create_optimizer(Sys0, traj, md_engine=options0['run']['md_engine'], 
    steps_equil=options0['run']['steps_equil'], 
    steps_prod=options0['run']['steps_prod'], 
    steps_stride=options0['run']['steps_stride'], 
    ElecSys=ElecSys)
Opt0.MinReweightFrac = 0.25
StageCoefsA = [100.0,100.0]
UDist0 = Opt0.AddPenalty(Az, Az_target, MeasureScale = 1., Coef=1.e-80)


Opt1 = optimizer.create_optimizer(Sys1, traj, md_engine=options1['run']['md_engine'], 
    steps_equil=options1['run']['steps_equil'], 
    steps_prod=options1['run']['steps_prod'], 
    steps_stride=options1['run']['steps_stride'], 
    ElecSys=Sys1proxy)
Opt1.MinReweightFrac = 0.75

StageCoefsRee = [1.0,10.0,100.0]
UDist1 = Opt1.AddPenalty(Dist2, Dist2_target, MeasureScale = 1., Coef=1.e-80)
print('Opt1 penalties: {}'.format(Opt1.Penalties))

===== Making optimizer =====
bondljg
NMol: 3795
NAtom: 5266
NDOF: 15798
Box: [  4.37698516   4.37698498  10.76481668]
... using OpenMM engine
... use target histograms: True
...check fixed parameters status:
i,Fixed
--- bond_C2_C2 ---
0 True
1 True
--- bond_C2_SO4 ---
0 True
1 False
--- ljg_W_W ---
0 True
1 True
2 True
3 True
4 True
--- ljg_C2_C2 ---
0 True
1 True
2 True
3 True
4 True
--- ljg_C2_W ---
0 True
1 True
2 True
3 True
4 True
--- ljg_Na_Na ---
0 True
1 True
2 True
3 True
4 True
--- ljg_Cl_Cl ---
0 True
1 True
2 True
3 True
4 True
--- ljg_Na_Cl ---
0 True
1 True
2 True
3 True
4 True
--- ljg_Na_W ---
0 True
1 True
2 True
3 True
4 True
--- ljg_Cl_W ---
0 True
1 True
2 True
3 True
4 True
--- ljg_C2_SO4 ---
0 True
1 True
2 False
3 True
4 True
--- ljg_C2_Na ---
0 True
1 True
2 False
3 True
4 True
--- ljg_C2_Cl ---
0 True
1 True
2 False
3 True
4 True
--- ljg_SO4_SO4 ---
0 True
1 True
2 False
3 True
4 True
--- ljg_SO4_Na ---
0 True
1 True
2 False
3 True
4 True
--- ljg_SO4_Cl ---
0 Tr

## Run with Area-constraint!

In [ ]:
print('\n... now adding A-constraint')
Opt0.UpdateMode = 0 #0=both, 1=srel only, 2=bias only
Opt0.RunStages(StageCoefs = StageCoefsA, LagMult0=10.0)
dname='bondljg0_matchA'
FilePrefix = Opt0.FilePrefix
TempFilePrefix = Opt0.TempFilePrefix
try:
  os.mkdir(dname)
  os.system('cp {}* {}'.format(FilePrefix,dname))
  os.system('cp {}* {}'.format(TempFilePrefix,dname))
except:
  print('file moving commands failed')
Opt0.OutputPotentials(FilePrefix = 'final_matchA')


## Final CGMD Run

In [ ]:
print('\n===== Final CGMD run =====')
export_sim.set_params_from_file(Opt0.ModSys.ForceField, FilePrefix + '_ff.dat')
Opt0.OutputPotentials(FilePrefix = 'final')
Opt0.MakeModTrajFunc( Opt0.ModSys, 'final', 10000, 100000, ElecSys=ElecSys, StepsStride=100, Verbose=False )

